<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy

In [2]:
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [23]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


We will be counting our tokens in each call.

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

## Create database

We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

In [3]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [4]:
from sqlalchemy import Column, Integer, String, Table, Float

luminoso = Table(
    "luminoso",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("concept", String(20), nullable=False),
    Column("positive_sentiment", Float, nullable=False),
    Column("Docs", Integer, nullable=False),
)

In [5]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [6]:
observations_lumi = [
    [1, 'laptop', 0.64, 1435],
    [2, 'appearance', 0.95, 1327],
    [3, 'smooth', 0.97, 601],
    [4, 'keyboard', 0.66, 687],
    [5, 'packaging protection', 0.83, 513],
    [6, 'screen', 0.55, 1320],
    [7, 'packaging', 0.80, 958],
    [8, 'computer', 0.66, 1454],
    [9, 'notebook', 0.67, 481],
    [10, 'satisfied', .99, 433],
]

In [7]:
from sqlalchemy import insert

def insert_obs_lumi(obs):
    stmt = insert(luminoso).values(
    obs_id=obs[0],
    concept=obs[1],
    positive_sentiment=obs[2],
    Docs=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [8]:
for obs in observations_lumi:
    insert_obs_lumi(obs)

We are installing the `langchain_experimental` library here, since the `SQLDatabaseChain` is located there. This might be changed in the future and moved into official `langchain` library.

In [9]:
!pip install langchain_experimental -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 4.5 MB/s eta 0:00:00


In [10]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

NameError: name 'llm' is not defined

## Agent types

### Agent type #1: Zero Shot React

In this first example we will use slightly different type of agent - SQL Agent which can be instantiated with it's own method `create_sql_agent`. Other agents will be instantiated in more generic way as we will see below in other examples.
<br><br>
This method uses *toolkit* instead of simple list of `tools`. You can read more about them in the [documentation](https://python.langchain.com/docs/modules/agents/toolkits/). For this use case, we will use `SQLDatabaseToolkit`.

As the name suggests, we will use this agent to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

Now we are ready to initialize the agent! We will use `verbose` in `True` so we can see what is our agent's 'thinking' process.

**Important Note:** *When interacting with agents it is really important to set the `max_iterations` parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.*

In [19]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

In [ ]:
result = count_tokens(
    agent_executor,
    "In the luminoso table what is the number of docs for the concept laptop "
)

Let's see our newly created agent in action! We will ask it a question that involves a math operation over the stock prices.

As always, let's see what the prompt is here:

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

Working with chatgpt 3.5 turbo (langchain tutorial)

In [11]:
%%capture
!pip install langchain_openai

In [12]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain.agents.agent import AgentExecutor

In [13]:
#need to figure out the agent_type...See above, don't understand what it does try to switch agen type here to see what happens

llm_chat = ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=OPENAI_API_KEY, temperature=0)

agent_executor_chat = create_sql_agent(llm_chat, db=db, agent_type="openai-tools", verbose=True)

In [24]:
#This agent gets the same result as above with llm model (doesn't work with llm_chat)...
agent_executor_chat_alt = create_sql_agent(llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [25]:
#figure how to only display the output
agent_executor_chat_alt.invoke(
    "In the luminoso table what is the number of docs for the concept laptop"
)



> Entering new SQL Agent Executor chain...
 I need to find the table that contains the concept laptop and then find the column that contains the number of docs.
Action: sql_db_list_tables
Action Input: ""luminosoNow that I have the table name, I can use sql_db_schema to get the schema and sample rows.
Action: sql_db_schema
Action Input: luminoso
CREATE TABLE luminoso (
	obs_id INTEGER NOT NULL, 
	concept VARCHAR(20) NOT NULL, 
	positive_sentiment FLOAT NOT NULL, 
	"Docs" INTEGER NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from luminoso table:
obs_id	concept	positive_sentiment	Docs
1	laptop	0.64	1435
2	appearance	0.95	1327
3	smooth	0.97	601
*/I can see that the column I need is called "Docs".
Action: sql_db_query_checker
Action Input: SELECT Docs FROM luminoso WHERE concept='laptop' SELECT Docs FROM luminoso WHERE concept='laptop' The query looks correct, so I can now use sql_db_query to get the result.
Action: sql_db_query
Action Input: SELECT Docs FROM luminoso WHERE concept='lapt

{'input': 'In the luminoso table what is the number of docs for the concept laptop',
 'output': '1435'}

In [15]:
res = agent_executor_chat.invoke(
    "In the luminoso table what is the number of docs for the concept laptop"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


luminoso
Invoking: `sql_db_schema` with `{'table_names': 'luminoso'}`



CREATE TABLE luminoso (
	obs_id INTEGER NOT NULL, 
	concept VARCHAR(20) NOT NULL, 
	positive_sentiment FLOAT NOT NULL, 
	"Docs" INTEGER NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from luminoso table:
obs_id	concept	positive_sentiment	Docs
1	laptop	0.64	1435
2	appearance	0.95	1327
3	smooth	0.97	601
*/
Invoking: `sql_db_query` with `{'query': "SELECT Docs FROM luminoso WHERE concept = 'laptop'"}`


[(1435,)]The number of documents for the concept "laptop" in the luminoso table is 1435.

> Finished chain.


creating a plain vanilla chatBot as agent

In [18]:
res['output']

'The number of documents for the concept "laptop" in the luminoso table is 1435.'

In [ ]:
llm_chain = prompt | llm

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.invoke(question)

another option:

In [ ]:
from langchain.llms import OpenAI  # Replace with your LLM provider (e.g., Bard)
from langchain_core.prompts import ChainGPTTemplate

# Define your LLM access (replace with your credentials)
llm = OpenAI(model="text-davinci-003", temperature=0)

# Create a simple prompt template with user input placeholder
prompt = ChainGPTTemplate(
    "Here's your question: {input}\nThe answer is: ",
    input_placeholder="{input}",
)

# Create the agent using ChatGPTLMChain (modify for your LLM provider)
agent = ChatGPTLMChain(llm=llm, prompt_template=prompt)

# Example usage
user_query = "What is the capital of France?"
response = agent.run(input=user_query)

print(f"User Query: {user_query}")
print(f"LLM Response: {response}")


Suffice it to say for now that **the LLM now has the ability to 'reason' on how to best use tools** to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to [this](https://arxiv.org/pdf/2205.00445.pdf) paper.

### Agent type #2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

We will use the math tool in this example and load it as below:

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain. For more information on memory, please refer to the 3rd chapter of this series.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [ ]:
result = count_tokens(
    conversational_agent,
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 10,000, 8%, 5 years
Observation: Answer: 14693.280768000006
Thought: Do I need to use a tool? No
AI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.

> Finished chain.
Spent a total of 1246 tokens


As we can see below, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

Let's see what happens if we try to answer the question that is related to the previous one:

In [ ]:
result = count_tokens(
    conversational_agent,
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 15000, 8%, 5 years, compound interest
Observation: Answer: 22039.92115200001
Thought: Do I need to use a tool? No
AI: If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, we would have $22,039.92, which is $7,346.64 more than the previous scenario.

> Finished chain.
Spent a total of 1416 tokens


### Agent type #3: React Docstore

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [ ]:
count_tokens(docstore_agent, "What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (, ARK-ihm-EE-deez; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.

{'input': "What were Archimedes' last words?",
 'output': 'Do not disturb my circles'}

We will not print the prompt here because it is too large, but you can see it yourself if you want to (we know how to already).

In short, it contains several examples of the `Question` > `Thought` > `Action` > `Observation` loop, that include the `Search` and `Lookup` tools.

If you want to learn more about this approach [this](https://arxiv.org/pdf/2210.03629.pdf) is the paper for ReAct

### Agent type #4: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [ ]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

We will not interact with this agent because for that we would need a serpapi key. However, by checking out the prompt we can see a few examples of how it works:

In [ ]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

As we can see, the prompt is basically a series of many examples to show the LLM how to ask follow up questions to a search tool until it can get to the final answer.

And.. again [here](https://arxiv.org/pdf/2210.03350.pdf) you have the paper to dive deeper!

### Wrapping up

And that all for agents! There's many other things you can do with agents, just to name a few:
* Create your own custom agent
* Use them with many other tools (even custom ones)
* Trace every call an agent makes through a convinient UI interface

Check the how-to [guides](https://langchain.readthedocs.io/en/latest/modules/agents/how_to_guides.html) for more!

In [ ]:
%%capture
!pip install anthropic

In [ ]:
import anthropic

In [ ]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=CLAUDE_API_KEY,
)

message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "How are you today?"}
    ]
)

print(message.content)